In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [21]:
from src.inference import get_feature_store

In [26]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-06 18:05:50,170 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 18:05:50,183 INFO: Initializing external client
2025-03-06 18:05:50,184 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 18:05:50,939 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648
Fetching data from 2025-02-05 23:05:50.170345+00:00 to 2025-03-06 22:05:50.170345+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.94s) 


In [27]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-06 00:00:00+00:00,2,0
1,2025-02-06 01:00:00+00:00,2,0
2,2025-02-06 02:00:00+00:00,2,0
3,2025-02-06 03:00:00+00:00,2,0
4,2025-02-06 04:00:00+00:00,2,0
...,...,...,...
178183,2025-03-06 18:00:00+00:00,263,157
178184,2025-03-06 19:00:00+00:00,263,115
178185,2025-03-06 20:00:00+00:00,263,106
178186,2025-03-06 21:00:00+00:00,263,99


In [28]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178188 entries, 0 to 184379
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         178188 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  178188 non-null  int32                  
 2   rides               178188 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.1 MB


In [29]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [30]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178188 entries, 0 to 184379
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         178188 non-null  datetime64[us]
 1   pickup_location_id  178188 non-null  int32         
 2   rides               178188 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.1 MB


In [31]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

Skipping location_id 156: Not enough data to create even one window.
Skipping location_id 84: Not enough data to create even one window.
Skipping location_id 44: Not enough data to create even one window.
Skipping location_id 109: Not enough data to create even one window.
Skipping location_id 204: Not enough data to create even one window.
Skipping location_id 5: Not enough data to create even one window.


In [32]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,2,0,0,1,0,0,1,1,0,...,1,0,0,0,0,1,0,1,189,2025-03-04 00:00:00
1,1,0,0,1,0,1,1,0,1,0,...,0,2,0,0,0,0,0,0,197,2025-03-04 00:00:00
2,0,0,2,1,0,0,0,1,1,0,...,2,0,2,0,1,0,1,0,247,2025-03-04 01:00:00
3,0,0,0,1,1,1,4,1,2,1,...,0,1,0,0,0,1,0,1,179,2025-03-04 01:00:00
4,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,21,2025-03-03 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,150,2025-03-03 22:00:00
248,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,183,2025-03-03 22:00:00
249,0,0,3,0,1,0,0,0,1,0,...,2,0,0,0,0,0,0,0,254,2025-03-03 23:00:00
250,166,63,54,47,249,11,111,168,205,279,...,126,187,249,260,244,206,206,168,163,2025-03-03 23:00:00


In [33]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-06 18:06:15,962 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 18:06:15,969 INFO: Initializing external client
2025-03-06 18:06:15,969 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 18:06:16,726 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648
Fetching data from 2025-02-05 23:06:15.962400+00:00 to 2025-03-06 22:06:15.962400+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.49s) 
Skipping location_id 5: Not enough data to create even one window.
Skipping location_id 44: Not enough data to create even one window.
Skipping location_id 84: Not enough data to create even one window.
Skipping location_id 109: Not enough data to create even one window.
Skipping location_id 156: Not enough data to create even one window.
Skipping location_id 204: Not enough data to create even one window.


In [34]:
current_date

Timestamp('2025-03-06 23:06:15.962400+0000', tz='Etc/UTC')

In [35]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-06
1,0,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,3,2025-03-06
2,0,3,1,0,1,0,1,5,7,2,...,7,18,20,17,9,12,18,6,4,2025-03-06
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-06
4,2,1,1,0,0,1,2,0,4,3,...,4,1,8,5,3,1,1,1,7,2025-03-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,259,2025-03-06
248,2,1,0,2,0,0,0,1,0,0,...,0,0,2,1,1,0,1,0,260,2025-03-06
249,4,1,0,0,2,2,2,7,7,16,...,59,40,50,78,57,48,34,12,261,2025-03-06
250,2,2,1,0,6,17,71,157,174,161,...,169,178,163,98,56,40,42,9,262,2025-03-06


In [36]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 18:06:29,361 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 18:06:29,363 INFO: Initializing external client
2025-03-06 18:06:29,363 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 18:06:30,065 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648

In [37]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [38]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,1.0
...,...,...
247,259,0.0
248,260,0.0
249,261,15.0
250,262,18.0


In [39]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 239, 138, 237, 142, 161, 163, 162, 141, 170], dtype=int32)